<a href="https://colab.research.google.com/github/Tomawock/NLP_Attack/blob/main/models/studio_3/DDI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!pip install optuna 

In [29]:
import pickle
import tensorflow as tf
import pandas as pd
import optuna
import numpy as np
from sklearn.preprocessing import OneHotEncoder
from tensorflow import keras
from sklearn.utils.class_weight import compute_class_weight

In [2]:
path="Desktop\\DDI"

In [ ]:
path="/content/drive/Shareddrives/Deep Learning/datasets/DDI/"

PATH PER WINDOWDS



In [ ]:
dev = pd.read_csv(path+'\ddi2013-type\\dev.tsv', sep='\t')
test = pd.read_csv(path+'\ddi2013-type\\test.tsv', sep='\t')
train = pd.read_csv(path+'\ddi2013-type\\train.tsv', sep='\t')

data_sinonimi = pd.read_csv(path+"\ddi2013-type\\DDI_sinonimi_test.csv")
data_embedding = pd.read_csv(path+"\ddi2013-type\\DDI_embedding_test.csv")

data_embedding_train = pd.read_csv(path+"\ddi2013-type\\DDI_embedding.csv")
data_sinonimi_train = pd.read_csv(path+"\ddi2013-type\\DDI_sinonimi.csv")

print(test.shape)
print(data_sinonimi.shape)
print(test)

In [ ]:
dev = pd.read_csv(path+'ddi2013-type/dev.tsv', sep='\t')
test = pd.read_csv(path+'ddi2013-type/test.tsv', sep='\t')
train = pd.read_csv(path+'ddi2013-type/train.tsv', sep='\t')

data_sinonimi = pd.read_csv(path+"ddi2013-type/DDI_sinonimi_test.csv")
data_embedding = pd.read_csv(path+"ddi2013-type/DDI_embedding_test.csv")

data_embedding_train = pd.read_csv(path+"ddi2013-type/DDI_embedding.csv")

print(test.shape)
print(data_sinonimi.shape)
print(test)

In [ ]:
test_study_1 = pd.concat([data_sinonimi, data_embedding, test], ignore_index=True)
train_study_1 = pd.concat([data_sinonimi_train, data_embedding_train, train], ignore_index=True)
print (test_study_1.shape)
print (train_study_1.shape)

In [ ]:
print(train.shape)
print(data_sinonimi.shape)
print(data_embedding.shape)

In [ ]:
print(train.label.value_counts())
print(data_sinonimi.label.value_counts())
print(data_embedding.label.value_counts())

In [5]:
with open(path+"\word2index.pkl", 'rb') as output:
  w2i = pickle.load(output)
with open(path+"\embedding_matrix.pkl", 'rb') as output:
  embedding_matrix = pickle.load(output)

Set up per allenamento Modello

In [6]:
categories = [['DDI-false', 'DDI-mechanism', 'DDI-effect', 'DDI-advise','DDI-int']]

my_text_to_word_sequence = lambda sen: keras.preprocessing.text.text_to_word_sequence(sen,
                                                                                      filters='!"#&()*+,-./:;<=>?[\\]^_`\'{|}~\t\n',
                                                                                      lower=True)

TRAINSET

In [7]:
five_hot_train = OneHotEncoder(sparse=False, categories=categories).fit_transform(
  train_study_1.label.to_numpy().reshape(-1, 1)
)

In [8]:
sentences_train = [my_text_to_word_sequence(sentence) for sentence in train_study_1['sentence']]

TESTSET

In [9]:
five_hot_test = OneHotEncoder(sparse=False, categories=categories).fit_transform(
  test_study_1.label.to_numpy().reshape(-1, 1)
)

In [10]:
sentences_test = [my_text_to_word_sequence(sentence) for sentence in test_study_1['sentence']]

ORIGINALE

In [11]:
five_hot_orig = OneHotEncoder(sparse=False, categories=categories).fit_transform(
  test.label.to_numpy().reshape(-1, 1)
)

In [12]:
sentences_orig = [my_text_to_word_sequence(sentence) for sentence in test['sentence']]

SINONIMI

In [13]:
five_hot_sin = OneHotEncoder(sparse=False, categories=categories).fit_transform(
  data_sinonimi.label.to_numpy().reshape(-1, 1)
)

In [14]:
sentences_sin = [my_text_to_word_sequence(sentence) for sentence in data_sinonimi['sentence']]

EMBEDDING

In [15]:
five_hot_emb = OneHotEncoder(sparse=False, categories=categories).fit_transform(
  data_embedding.label.to_numpy().reshape(-1, 1)
)

In [16]:
sentences_emb = [my_text_to_word_sequence(sentence) for sentence in data_embedding['sentence']]

Estrai la massima dimensione dell'input in base ai vari dataset considerati

In [17]:
max_index, max = (-1, -1)
for i, sentence in enumerate(sentences_train):
  max_index, max = (i, len(sentence)) if len(sentence) > max else (max_index, max)
for i, sentence in enumerate(sentences_test):
  max_index, max = (i, len(sentence)) if len(sentence) > max else (max_index, max)
for i, sentence in enumerate(sentences_orig):
  max_index, max = (i, len(sentence)) if len(sentence) > max else (max_index, max)
for i, sentence in enumerate(sentences_sin):
  max_index, max = (i, len(sentence)) if len(sentence) > max else (max_index, max)
for i, sentence in enumerate(sentences_emb):
  max_index, max = (i, len(sentence)) if len(sentence) > max else (max_index, max)
  
print(f'Il massimo è {max}')

Il massimo è 92


Crao i vari embedding per tutti i dataset, quest'operazione e pesante 

In [18]:
embedded_trainset = np.zeros(shape=(len(sentences_train), max, 300))
for i, sentence in enumerate(sentences_train):
  for j, word in enumerate(sentence):
    try:
      embedded_trainset[i, j, :] = embedding_matrix[w2i[word]]
    except KeyError:
      pass

In [19]:
embedded_origin = np.zeros(shape=(len(sentences_orig), max, 300))
for i, sentence in enumerate(sentences_orig):
  for j, word in enumerate(sentence):
    try:
      embedded_origin[i, j, :] = embedding_matrix[w2i[word]]
    except KeyError:
      pass

In [20]:
embedded_testset = np.zeros(shape=(len(sentences_test), max, 300))
for i, sentence in enumerate(sentences_test):
  for j, word in enumerate(sentence):
    try:
      embedded_testset[i, j, :] = embedding_matrix[w2i[word]]
    except KeyError:
      pass

In [21]:
embedded_sin = np.zeros(shape=(len(sentences_sin), max, 300))
for i, sentence in enumerate(sentences_sin):
  for j, word in enumerate(sentence):
    try:
      embedded_sin[i, j, :] = embedding_matrix[w2i[word]]
    except KeyError:
      pass

In [22]:
embedded_emb = np.zeros(shape=(len(sentences_emb), max, 300))
for i, sentence in enumerate(sentences_emb):
  for j, word in enumerate(sentence):
    try:
      embedded_emb[i, j, :] = embedding_matrix[w2i[word]]
    except KeyError:
      pass

Carica optuna results e inizializza il modello, oppure salva il modello oppure carica solo i pesi del modello 

In [23]:
best_params = optuna.load_study(study_name="DDI", storage="sqlite:///"+path+"\ddi2013-type\\optuna_ddi_studio_0.db").best_params

In [24]:
print(f'{best_params}')

{'batch_size': 89, 'dropout': 0.63, 'units': 81}


In [25]:
model = keras.Sequential()
model.add(keras.layers.Input(shape=(max, 300)))
model.add(keras.layers.Bidirectional(layer=keras.layers.LSTM(units=best_params['units'],
                                                             recurrent_dropout=best_params['dropout'],
                                                             activation='tanh')))

model.add(keras.layers.Dense(5, activation='softmax'))
model.compile(loss='categorical_crossentropy',
              optimizer=keras.optimizers.Adam(learning_rate=0.001),
              metrics=['accuracy'])

In [32]:
#train the model
result = model.fit(embedded_trainset,
                   five_hot_train,
                   epochs=100,
                   batch_size=best_params['batch_size'],
                   callbacks=[keras.callbacks.EarlyStopping(monitor='loss',
                                                            patience=10)])

Epoch 1/100
633/633 [==============================] - 137s 216ms/step - loss: 0.5420 - accuracy: 0.8422
Epoch 2/100
633/633 [==============================] - 141s 223ms/step - loss: 0.4382 - accuracy: 0.8456
Epoch 3/100
633/633 [==============================] - 138s 218ms/step - loss: 0.4003 - accuracy: 0.8498
Epoch 4/100
633/633 [==============================] - 139s 220ms/step - loss: 0.3754 - accuracy: 0.8541
Epoch 5/100
633/633 [==============================] - 140s 221ms/step - loss: 0.3569 - accuracy: 0.8578
Epoch 6/100
633/633 [==============================] - 142s 224ms/step - loss: 0.3387 - accuracy: 0.8637
Epoch 7/100
633/633 [==============================] - 141s 223ms/step - loss: 0.3232 - accuracy: 0.8684
Epoch 8/100
633/633 [==============================] - 141s 222ms/step - loss: 0.3069 - accuracy: 0.8742
Epoch 9/100
633/633 [==============================] - 141s 222ms/step - loss: 0.2940 - accuracy: 0.8792
Epoch 10/100
633/633 [==============================] -

In [33]:
#save the model
model.save_weights(path+'\ddi2013-type\\DDI_w_studio_3.h5')

In [ ]:
#load only the w of the model, the model must be already executed
model.load_weights(path+'\ddi2013-type\\DDI_w_studio_3.h5')

EVALUATE ALL DATASET

In [34]:
result_base=model.evaluate(embedded_trainset, five_hot_train, batch_size=best_params['batch_size'],)
print(f'DATASET ORIGINARIO{result_base}')

633/633 [==============================] - 43s 68ms/step - loss: 0.0288 - accuracy: 0.9906
DATASET ORIGINARIO[0.028838587924838066, 0.9906455874443054]


In [35]:
result_base=model.evaluate(embedded_testset, five_hot_test, batch_size=best_params['batch_size'],)
print(f'DATASET TEST{result_base}')

195/195 [==============================] - 12s 59ms/step - loss: 0.9017 - accuracy: 0.8169
DATASET TEST[0.9017105102539062, 0.8168720602989197]


In [36]:
result_base=model.evaluate(embedded_origin, five_hot_orig, batch_size=best_params['batch_size'],)
print(f'DATASET TEST{result_base}')

65/65 [==============================] - 4s 58ms/step - loss: 0.8376 - accuracy: 0.8228
DATASET TEST[0.8375811576843262, 0.822773814201355]


In [37]:
result_base=model.evaluate(embedded_sin, five_hot_sin, batch_size=best_params['batch_size'],)
print(f'DATASET SINONIMI{result_base}')

65/65 [==============================] - 4s 59ms/step - loss: 0.9645 - accuracy: 0.8136
DATASET SINONIMI[0.9645338654518127, 0.8135740160942078]


In [38]:
result_base=model.evaluate(embedded_emb, five_hot_emb, batch_size=best_params['batch_size'],)
print(f'DATASET EMBEDDING{result_base}')

65/65 [==============================] - 4s 59ms/step - loss: 0.9030 - accuracy: 0.8143
DATASET EMBEDDING[0.9030170440673828, 0.8142683506011963]


tali test sono stati effettuati su windows, eventuali path necessitano di essere adattati, se effettuati su sistema operativo unix like.

